In [599]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
import numpy as np

import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException


In [304]:
#step 1 
#Get url for product in each category
driver = webdriver.Chrome('/Users/bestmacbook/Downloads/chromedriver')
productcat = ['face-wash-facial-cleanser', 'facial-toner-skin-toner', 'moisturizer-skincare']
df = pd.DataFrame(columns=['Category', 'URL'])

for cat in productcat:
    driver.get('https://www.sephora.com/shop/'+cat+'?pageSize=300')
    time.sleep(1)

    elem = driver.find_element_by_tag_name("body")

    no_of_pagedowns = 10
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1
    
    pi = driver.find_elements_by_class_name("css-ix8km1")

    producturl = []
    for a in pi:
        subURL = a.get_attribute('href')
        producturl.append(subURL)
    
    dic = {'Category': cat, 'URL': producturl}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

In [305]:
df.groupby('Category').count()

,URL
Category,
face-wash-facial-cleanser,96
facial-toner-skin-toner,96
moisturizer-skincare,96


In [306]:
#clean product url 
newurl = []
for i in df.URL:
    newi = i.split('?')[0]
    newurl.append(newi)
df.URL = newurl

In [307]:
df.to_csv('url.csv')

In [555]:
#step 2
#scrape product brands, names, ranks, prices, descriptions, ingredients and skin types
df = pd.read_csv('./url.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df_info = pd.DataFrame(columns=['brand', 'name', 'rating', 'price',
                                'descriptions', 'ingredients','pic'])
df = pd.concat([df, df_info], axis = 1)

In [609]:
#####samples#####

In [630]:

driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.sephora.com/product/legend-P375713')
time.sleep(2)


item = driver.find_element_by_class_name('css-a1jw00').text.split('\n')
br = item[0]
na = item[1]
pr = driver.find_element_by_class_name('css-14hdny6').text


de = driver.find_element_by_class_name('css-1rny024').text

elem = driver.find_element_by_tag_name("body")
elem.send_keys(Keys.PAGE_DOWN)
time.sleep(2)

driver.find_element_by_id('tab1').click()
ing = driver.find_element_by_id('tabpanel1').text
time.sleep(2)    
no_of_pagedowns = 3

while no_of_pagedowns:
    elem = driver.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1

try:
    ra = driver.find_element_by_class_name('css-1eqf5yr').text
    ra = float(ra.split(' / ')[0])
except NoSuchElementException:
    df_info.rating[0] = np.nan
time.sleep(1)
driver.find_element_by_class_name('css-1juuxmf').click()
time.sleep(1)
pi = driver.find_element_by_class_name('css-1glglyy').get_attribute('src')



In [631]:
pd.DataFrame({'brand':br, 'name':na, 'rating':ra, 'price':pr,'descriptions':de, 'ingredients':ing,'pic':pi},index=[0])

,brand,name,rating,price,descriptions,ingredients,pic
0,MONTBLANC,Legend,4.8,$92.00,The Montblanc Legend man is confident and char...,"Alcohol Denat. (Sd Alcohol 39-C), Water, Fragr...",https://www.sephora.com/productimages/sku/s145...


In [ ]:
DataFrame({'brand':br, 'name':na, 'rating':ra, 'price':pr,'descriptions':de, 'ingredients':ing,'pic':pi},index=[0])

In [569]:
driver = webdriver.Chrome('./chromedriver')
for i in range(len(df.URL)):
    driver.get(df.URL[i])
    time.sleep(5)

    #product brand and name
    item = driver.find_element_by_class_name('css-a1jw00').text.split('\n')
    df.brand[i] = item[0]
    df.name[i] = item[1]
    
    #price
    df.price[i] = driver.find_element_by_class_name('css-14hdny6').text
    
    #descriptions
    df.descriptions[i]= driver.find_element_by_class_name('css-1rny024').text
    
    #one page down
    elem = driver.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    #ingredient
    #we need to click ingredient tab in product infomation table to get the text
    driver.find_element_by_id('tab2').click()
    df.ingredients[i] = driver.find_element_by_id('tabpanel2').text
    
    #scoll down so the website will show rating and review box
    no_of_pagedowns = 3
    while no_of_pagedowns:
        elem = driver.find_element_by_tag_name("body")
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        no_of_pagedowns-=1
    
    #rating 
    #there may be some product don't have ratings 
    try:
        ra = driver.find_element_by_class_name('css-1eqf5yr').text
        df.rating[i] = float(ra.split(' / ')[0])
    except NoSuchElementException:
        df.rating[i] = np.nan
    
    #pic
    driver.find_element_by_class_name('css-1juuxmf').click()
    time.sleep(1)
    df.pic[i] = driver.find_element_by_class_name('css-1glglyy').get_attribute('src')
    
    time.sleep(0.5)
     

In [608]:
df.head()

,Category,URL,brand,name,rating,price,descriptions,ingredients,pic
0,face-wash-facial-cleanser,https://www.sephora.com/product/soy-face-clean...,FRESH,Soy Face Cleanser,4.4,$38.00,"What it is: A bestselling, gently powerful fac...",-Amino Acid-rich Soy Proteins: Help maintain s...,https://www.sephora.com/productimages/sku/s487...
1,face-wash-facial-cleanser,https://www.sephora.com/product/kale-spinach-g...,YOUTH TO THE PEOPLE,Superfood Antioxidant Cleanser,4.3,$36.00,What it is: A daily green juice cleanse for yo...,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...",https://www.sephora.com/productimages/sku/s186...
2,face-wash-facial-cleanser,https://www.sephora.com/product/green-clean-ma...,FARMACY,Green Clean Makeup Removing Cleansing Balm,4.6,$34.00,What it is: An award-winning makeup remover an...,-Sunflower and Ginger Root Oils: Melt even stu...,https://www.sephora.com/productimages/sku/s189...
3,face-wash-facial-cleanser,https://www.sephora.com/product/sugar-strawber...,FRESH,Sugar Strawberry Exfoliating Face Wash,4.5,$33.00,What it is: A gentle exfoliating face cleanser...,-Blend of White Sugar Crystals: Help smooth an...,https://www.sephora.com/productimages/sku/s222...
4,face-wash-facial-cleanser,https://www.sephora.com/product/purity-made-si...,PHILOSOPHY,Purity Made Simple Cleanser,4.5,$24.00,"What it is: A face wash for easy, one-step cle...",-Sodium Lauroamphoacetate: Acts as a mild surf...,https://www.sephora.com/productimages/sku/s407...


In [607]:
df.to_csv('product_info_raw.csv')